# GPX data analysis (multiple files)

This notebook contains some Python test code to analyze and visualize GPX data from a multiple files. GPX (GPS Exchange Format) is an XML based file format for GPS tracks, and is widely used by dozens of software programs and web services for GPS data analysis and visualization. 

Data manipulation is done using [gpxpy](https://github.com/tkrajina/gpxpy), a GPX parser for Python, and the [NumPy](http://www.numpy.org/) and [Pandas](https://pandas.pydata.org/) data analysis packages for Python. Visualization is done using [gmplot](https://github.com/vgm64/gmplot) and [folium](https://github.com/python-visualization/folium).

## Prerequisites:

In [1]:
import datetime
import glob # module to find all pathnames matching a specified pattern
import os

import gpxpy
import numpy  as np
import pandas as pd
import gmplot
import folium
from folium import plugins

%load_ext watermark
%watermark -a "Author: gmalim" 
print("")
%watermark -u -n
print("")
%watermark -v -p gpxpy,numpy,pandas,gmplot,folium
print("")
%watermark -m

Author: gmalim

last updated: Wed Jul 18 2018

CPython 3.6.5
IPython 6.4.0

gpxpy n
numpy 1.14.5
pandas 0.23.3
gmplot n
folium 0.5.0+145.gb7b31aa

compiler   : GCC 4.2.1 Compatible Apple LLVM 8.0.0 (clang-800.0.42.1)
system     : Darwin
release    : 15.6.0
machine    : x86_64
processor  : i386
CPU cores  : 2
interpreter: 64bit


## Load data:

Create function to load list of GPX files:

In [2]:
def load_run_data(gpx_path, filter="*"):
    """
    Loop over files, tracks, segments and points.
    """
    
    gpx_files = glob.glob(os.path.join(gpx_path, filter + ".gpx"))
    
    gpx_points = []
    
    total_track_length   = 0
    total_track_duration = 0
    
    trainingrun_id = 1

    for file_idx, gpx_file in enumerate(gpx_files):
    
        print("--> Processing file", file_idx)
        
        gpx = gpxpy.parse(open(gpx_file, 'r'))
        
        for track_idx, track in enumerate(gpx.tracks):
            
            #print("---> Processing track", track_idx)
            
            track_name     = track.name
            track_length   = track.length_3d()
            track_duration = track.get_duration()

            total_track_length   += track_length
            total_track_duration += track_duration
                        
            if (track_name == 'Training run'):
                track_name += ' {}'.format(trainingrun_id) 
                trainingrun_id += 1
               
            #track_name += ' ({:.0f} km)'.format(track.length_3d()/1e3)
            
            for seg_idx, segment in enumerate(track.segments):
                
                #print("----> Processing segment", seg_idx)
                
                #segment_length = segment.length_3d()
                
                for point_idx, point in enumerate(segment.points):
                    
                    #print("----> Processing point", point_idx)
                    
                    gpx_points.append([
                                       track_name,
                                       point.time,
                                       point.latitude, 
                                       point.longitude, 
                                       #point.elevation, 
                                       #segment.get_speed(point_idx)
                                      ])
                    
    return gpx_points, total_track_length, total_track_duration

Load list of GPX files and parse data to dataframe:

In [3]:
gpx_points, total_track_length, total_track_duration = load_run_data(gpx_path='./data/', filter="*")

print('')
print('Total distance as summed between points in track = {:6.2f} miles'.format(total_track_length*0.000621371))
print('Total distance as summed between points in track = {:6.2f} km'   .format(total_track_length*0.001))
print('Total time     as summed between points in track = {:6.2f} hours'.format(total_track_duration/3600))

column_list = [
               'Track_Name',
               'Point_Time', 
               'Point_Latitude',
               'Point_Longitude', 
               #'Point_Elevation', 
               #'Point_Speed'
              ]

df = pd.DataFrame(gpx_points, columns=column_list)

--> Processing file 0
--> Processing file 1
--> Processing file 2
--> Processing file 3
--> Processing file 4
--> Processing file 5
--> Processing file 6

Total distance as summed between points in track =  72.33 miles
Total distance as summed between points in track = 116.40 km
Total time     as summed between points in track =  13.34 hours


Optional: Save dataframe using pickle:

In [4]:
df.to_pickle("alldata.pkl")

print(df.shape)
print(df.head())

(46651, 4)
             Track_Name          Point_Time  Point_Latitude  Point_Longitude
0  Bay to Breakers 2016 2016-05-15 15:07:53       37.790538      -122.393670
1  Bay to Breakers 2016 2016-05-15 15:07:54       37.790533      -122.393673
2  Bay to Breakers 2016 2016-05-15 15:07:55       37.790526      -122.393684
3  Bay to Breakers 2016 2016-05-15 15:07:56       37.790527      -122.393673
4  Bay to Breakers 2016 2016-05-15 15:07:57       37.790519      -122.393669


Optional: Load dataframe from pickled file:

In [5]:
df = pd.read_pickle("alldata.pkl")

print(df.shape)
print(df.head())

(46651, 4)
             Track_Name          Point_Time  Point_Latitude  Point_Longitude
0  Bay to Breakers 2016 2016-05-15 15:07:53       37.790538      -122.393670
1  Bay to Breakers 2016 2016-05-15 15:07:54       37.790533      -122.393673
2  Bay to Breakers 2016 2016-05-15 15:07:55       37.790526      -122.393684
3  Bay to Breakers 2016 2016-05-15 15:07:56       37.790527      -122.393673
4  Bay to Breakers 2016 2016-05-15 15:07:57       37.790519      -122.393669


Optional: Select data according to time:

In [6]:
df['Point_Time'] = pd.to_datetime(df['Point_Time'])

start_date = pd.Timestamp(2015,1,1)   
end_date   = pd.Timestamp(2018,1,1)   

mask = ((df['Point_Time'] > start_date) & (df['Point_Time'] <= end_date))

df = df.loc[mask]

print(df.shape)
print(df.head())

(46651, 4)
             Track_Name          Point_Time  Point_Latitude  Point_Longitude
0  Bay to Breakers 2016 2016-05-15 15:07:53       37.790538      -122.393670
1  Bay to Breakers 2016 2016-05-15 15:07:54       37.790533      -122.393673
2  Bay to Breakers 2016 2016-05-15 15:07:55       37.790526      -122.393684
3  Bay to Breakers 2016 2016-05-15 15:07:56       37.790527      -122.393673
4  Bay to Breakers 2016 2016-05-15 15:07:57       37.790519      -122.393669


## Visualization:

Create heatmap using **gmplot**:

In [7]:
map_center_lat = df.Point_Latitude .mean() + 0.01
map_center_lon = df.Point_Longitude.mean()

print("map_center_lat = {:8.3f} degrees".format(map_center_lat))
print("map_center_lon = {:8.3f} degrees".format(map_center_lon))

gmap = gmplot.GoogleMapPlotter(map_center_lat, map_center_lon, 13) # lat & lon of map center and map zoom level

gmap.heatmap(df['Point_Latitude'], df['Point_Longitude'], maxIntensity=100)

gmap.draw("gmplot_heatmap.html")

map_center_lat =   37.796 degrees
map_center_lon = -122.439 degrees


In [8]:
%%HTML
<iframe width="100%" height="700" src="gmplot_heatmap.html"></iframe>

Create heatmap using **folium**:

In [9]:
# List comprehension to make out list of lists needed for folium.plugins.HeatMap class:

heat_data = [[row['Point_Latitude'],row['Point_Longitude']] for index, row in df.iterrows()]

In [10]:
# Create basemap:

fmap = folium.Map(location=[map_center_lat, map_center_lon], zoom_start = 13, control_scale = True, tiles=None)

# Add tileset layers to map: 

# Option 1 - Use built-in tilesets:

builtin_tilesets = [#'OpenStreetMap', 
                    #'Mapbox Bright', 
                    #'Mapbox Control Room', 
                    'CartoDB Positron', 
                    'CartoDB Dark_Matter',
                    #'Stamen Toner', 
                    #'Stamen Watercolor',
                    #'Stamen Terrain'
                   ]

for tileset in builtin_tilesets:
    fmap.add_tile_layer(tiles=tileset, name=tileset) #, attr=tileset)

# Option 2 - Use tileset servers (See http://leaflet-extras.github.io/leaflet-providers/preview/):

#fmap.add_tile_layer(tiles='https://{s}.tile.opentopomap.org/{z}/{x}/{y}.png', name='OpenTopoMap', attr='OpenTopoMap')

url_base = 'http://server.arcgisonline.com/ArcGIS/rest/services/'

mapservernames = [#'Ocean_Basemap', 
                  #'USA_Topo_Maps', 
                  #'NatGeo_World_Map',
                  'World_Imagery', 
                  #'World_Physical_Map', 
                  #'World_Street_Map', 
                  #'World_Terrain_Base', 
                  #'World_Topo_Map'
                 ]

for msn in mapservernames:
    tileset = url_base + msn + '/MapServer/tile/{z}/{y}/{x}'
    fmap.add_tile_layer(tiles=tileset, name=msn, attr=msn)

# Add fullscreen button:

plugins.Fullscreen(
    position='topleft',
    title='Enter fullscreen-mode',
    title_cancel='Exit fullscreen-mode',
    force_separate_button=True).add_to(fmap)

# Add button to enabling/disabling scrolling - DOESN'T WORK?:

# plugins.ScrollZoomToggler().add_to(fmap)

# Add latitude-longitude popup click-tool:

fmap.add_child(folium.LatLngPopup())

# Add HeatMap as FeatureGroup:

hm = plugins.HeatMap(heat_data, radius=5, blur=0.25, min_opacity=0.15, max_val=1e2, max_zoom=13)

h = folium.FeatureGroup(name='Heat Map')
h.add_child(hm)
fmap.add_child(h)

# Split dataframe by trackname and add each track & minute-spread points along the track as a FeatureGroup:

grouped = df.groupby(['Track_Name'])

for track_name in df.Track_Name.unique():
    
    dftmp = grouped.get_group(track_name)

    print("--> Processing", track_name)
    #print(dftmp.shape)

    fg = folium.FeatureGroup(name=track_name, show=False)

    polylinepoints = []
    for lat, lon in zip(dftmp['Point_Latitude'], dftmp['Point_Longitude']):
        polylinepoints.append(tuple([lat,lon]))

    pl = folium.PolyLine(locations=polylinepoints, color="black", weight=2, opacity=0.5)
    fg.add_child(pl)
    
    npoints = len(dftmp)
    
    for i, coord in enumerate(dftmp[['Point_Latitude','Point_Longitude']].values):
        
        if (i == 0):
            m = folium.Marker(location=[coord[0],coord[1]], 
                              icon=folium.Icon(color='green', prefix='fa', icon='hourglass-start'))
            fg.add_child(m)
        
        if (i == npoints-1):
            m = folium.Marker(location=[coord[0],coord[1]], 
                              icon=folium.Icon(color='red', prefix='fa', icon='hourglass-end'))
            fg.add_child(m)
        
        if (i%60 != 0):
            continue
            
        cm = folium.CircleMarker(location=[coord[0],coord[1]], radius=1, color='magenta')
        fg.add_child(cm)
                
    fmap.add_child(fg)

# Add layer-control button:
    
lc = folium.LayerControl()
fmap.add_child(lc)
    
# Save map:
    
fmap.save('folium_heatmap.html')

--> Processing Bay to Breakers 2016
--> Processing Bay to Breakers 2017
--> Processing SF Marathon 2015
--> Processing Training run 1
--> Processing Training run 2
--> Processing Training run 3
--> Processing Training run 4


In [11]:
%%HTML
<iframe width="100%" height="700" src="folium_heatmap.html"></iframe>